In [1]:
__author__      = "Federico Ranaldi"

In [ ]:
import numpy as np
import pandas as pd
import string
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
df=pd.read_csv('data.csv')

In [3]:
df.head()

,Unnamed: 0,Unnamed: 0.1,text,author
0,0,0.0,Who is the happy Warrior ? Who is he That ever...,William Wordsworth
1,1,1.0,"wish to be ? —It is the generous Spirit , who ...",William Wordsworth
2,2,2.0,"the tasks of real life , hath wrought Upon the...",William Wordsworth
3,3,3.0,thought : Whose high endeavours are an inward ...,William Wordsworth
4,4,4.0,"always bright ; Who , with a natural instinct ...",William Wordsworth


In [5]:
df["author"].unique()

array(['William Wordsworth', 'Percy Bysshe Shelley',
       'William Shakespeare', 'Alfred, Lord Tennyson',
       'Algernon Charles Swinburne', 'Anonymous', 'Walt Whitman',
       'Edgar Allan Poe', 'Howard Phillips Lovecraft',
       'Mary Wollstonecraft Shelley'], dtype=object)

In [6]:
df.groupby("author").count()

,text
author,
"Alfred, Lord Tennyson",2318
Algernon Charles Swinburne,2155
Anonymous,2300
Edgar Allan Poe,2500
Howard Phillips Lovecraft,2500
Mary Wollstonecraft Shelley,2500
Percy Bysshe Shelley,2301
Walt Whitman,2251
William Shakespeare,2143


In [8]:
df.index=range(len(df))

In [9]:
df.groupby("author").count()

,text
author,
Edgar Allan Poe,2500
Howard Phillips Lovecraft,2500
Mary Wollstonecraft Shelley,2500
Percy Bysshe Shelley,2301
William Shakespeare,2143


**WORD-TOKENIZATION**

In [10]:
nltk.download('punkt')

df['tokenized_sents'] = df.apply(lambda row: nltk.word_tokenize(row["text"]), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


**POS-TAGGIN(PART-OF-SPEECH TAGGING)**

In [11]:

nltk.download('averaged_perceptron_tagger')

df['tokenized_sents'] = df['tokenized_sents'].apply(lambda x: pos_tag(x))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [12]:
df['tokenized_sents']

0        [(I, PRP), (I, PRP), (weep, VBP), (for, IN), (...
1        [(our, PRP$), (tears, NNS), (Thaw, NNP), (not,...
2        [(,, ,), (sad, JJ), (Hour, NNP), (,, ,), (sele...
3        [(obscure, JJ), (compeers, NNS), (,, ,), (And,...
4        [(Died, NNP), (Adonais, NNP), (;, :), (till, V...
                               ...                        
11939    [(I, PRP), (need, VBP), (not, RB), (conjure, V...
11940    [(The, DT), (apprehension, NN), (,, ,), (that,...
11941    [(I, PRP), (will, MD), (not, RB), (live, VB), ...
11942    [(After, IN), (the, DT), (departure, NN), (of,...
11943    [(Reproach, NNP), (is, VBZ), (indeed, RB), (an...
Name: tokenized_sents, Length: 11944, dtype: object

**TO-LOWER-CASE**

In [13]:
def listOfLists(L):
  newL=[]
  for t in L:
    newL.append(list(t))
  return newL

def toLowerCase(L):
  for l in L:
    l[0]=l[0].lower()
  return L

#trasformazione da lista di tuple a lista di liste
df['tokenized_sents']=df['tokenized_sents'].apply(lambda x: listOfLists(x))

#trasformazione dei token in lower-case
df['tokenized_sents']=df['tokenized_sents'].apply(lambda x: toLowerCase(x))

In [14]:
df["tokenized_sents"]

0        [[i, PRP], [i, PRP], [weep, VBP], [for, IN], [...
1        [[our, PRP$], [tears, NNS], [thaw, NNP], [not,...
2        [[,, ,], [sad, JJ], [hour, NNP], [,, ,], [sele...
3        [[obscure, JJ], [compeers, NNS], [,, ,], [and,...
4        [[died, NNP], [adonais, NNP], [;, :], [till, V...
                               ...                        
11939    [[i, PRP], [need, VBP], [not, RB], [conjure, V...
11940    [[the, DT], [apprehension, NN], [,, ,], [that,...
11941    [[i, PRP], [will, MD], [not, RB], [live, VB], ...
11942    [[after, IN], [the, DT], [departure, NN], [of,...
11943    [[reproach, NNP], [is, VBZ], [indeed, RB], [an...
Name: tokenized_sents, Length: 11944, dtype: object

**STOPWORDS-REMOVAL**

In [15]:
nltk.download('stopwords')

stop_list = stopwords.words('english')+list(string.punctuation)+[" "]+[""] #noise removal:insieme alle stopwords viene eliminata la punteggiatura

print(stop_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [16]:
#RIMOZIONE DELLE STOPWORDS (comprende rimozione dei rumori)
#rimuove troppe parole!!!
#rimuove gli elementi della sottolista b dagli elementi della sottolista a
def removeSublist(a,b):
  for el in b:
    a.remove(el)

#df['tokenized_sents'].apply(lambda x: removeSublist(x,[couple for couple in x if not(set(couple[0]).isdisjoint(stop_list))]))

**NOISE-REMOVAL**

In [17]:
#solo rimozione di rumori(punteggiatura e spazi vuoti)

def removeSublist(a,b):
  for el in b:
    a.remove(el)

noises_list=list(string.punctuation)+[" "]+[""]

df['tokenized_sents'].apply(lambda x: removeSublist(x,[couple for couple in x if not(set(couple[0]).isdisjoint(noises_list))]))

0        None
1        None
2        None
3        None
4        None
         ... 
11939    None
11940    None
11941    None
11942    None
11943    None
Name: tokenized_sents, Length: 11944, dtype: object

In [18]:
df["tokenized_sents"]

0        [[i, PRP], [i, PRP], [weep, VBP], [for, IN], [...
1        [[our, PRP$], [tears, NNS], [thaw, NNP], [not,...
2        [[sad, JJ], [hour, NNP], [selected, VBN], [fro...
3        [[obscure, JJ], [compeers, NNS], [and, CC], [t...
4        [[died, NNP], [adonais, NNP], [till, VB], [the...
                               ...                        
11939    [[i, PRP], [need, VBP], [not, RB], [conjure, V...
11940    [[the, DT], [apprehension, NN], [that, IN], [r...
11941    [[i, PRP], [will, MD], [not, RB], [live, VB], ...
11942    [[after, IN], [the, DT], [departure, NN], [of,...
11943    [[reproach, NNP], [is, VBZ], [indeed, RB], [an...
Name: tokenized_sents, Length: 11944, dtype: object

**LEMMATIZATION**

In [19]:
#Lemmatizzazione in base al Pos-Tag
#Attenzione funziona solo con #codice per POS-TAG

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 0

In [91]:
lemmatizer.lemmatize("going",pos=get_wordnet_pos("VBZ"))

'go'

In [22]:
#Lemmatizzazione in base al Pos-Tag
#Attenzione funziona solo con #codice per POS-TAG

from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import word_tokenize

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return 0

lemmatizer = WordNetLemmatizer()

def lemmatizeToken(L):
  for i in range(len(L)):
    if(get_wordnet_pos(L[i][1])!=0):
      L[i][0]=lemmatizer.lemmatize(L[i][0],pos=get_wordnet_pos(L[i][1]))
  return L

df["tokenized_sents"]=df['tokenized_sents'].apply(lambda x: lemmatizeToken(x))

In [23]:
df["tokenized_sents"]

0        [[i, PRP], [i, PRP], [weep, VBP], [for, IN], [...
1        [[our, PRP$], [tear, NNS], [thaw, NNP], [not, ...
2        [[sad, JJ], [hour, NNP], [select, VBN], [from,...
3        [[obscure, JJ], [compeer, NNS], [and, CC], [te...
4        [[died, NNP], [adonais, NNP], [till, VB], [the...
                               ...                        
11939    [[i, PRP], [need, VBP], [not, RB], [conjure, V...
11940    [[the, DT], [apprehension, NN], [that, IN], [r...
11941    [[i, PRP], [will, MD], [not, RB], [live, VB], ...
11942    [[after, IN], [the, DT], [departure, NN], [of,...
11943    [[reproach, NNP], [be, VBZ], [indeed, RB], [an...
Name: tokenized_sents, Length: 11944, dtype: object

CONCATENAZIONE DELLE COPPIE TOKEN-TAG OPPURE TOKEN-ENTITY

In [24]:

df["tokenized_sents"]=df["tokenized_sents"].apply(lambda x: ' '.join([el[0]+el[1] for el in x]))

In [25]:
df.index=range(len(df))

In [26]:
df.loc[970,"tokenized_sents"]

'everRB toTO sageVB orCC poetVB theseDT responseNNS giveVBN thereforeRB theDT nameNNS ofIN demonNNP'

# **VETTORIZZAZIONE CON TF-IDF**

Il TF-IDF è un algoritmo di vettorizzazione che converte in un formato numerico il nostro corpus facendo emergere termini specifici, pesando diversamente i termini molto rari o molto comuni in modo da assegnare loro un punteggio basso.
TF sta per term frequency, mentre IDF sta per inverse document frequency. 
Il valore TF-IDF legato ad una parola(o token) aumenta proporzionalmente al numero di volte che questa appare nel documento ed è compensato dal numero di documenti nel corpus che la contengono.Se una parola è contenuta in molti documenti allora è probabile che quella non sia una parola altamente specifica per il documento.
Nell'algorimo TF-IDF così come in BOW non viene codificato l'ordine dei token all'interno di un testo e ciò causa un ulteriore perdita di informazione oltre a quella generata dal preprocessing.
Mentre BOW si limita a codificare le parole contenute in un testo preprocessato TFIDF codifica l'informazione che descrive l'importanza di una parola all'interno di un testo che fa parte di un insieme di documenti su cui il modello verrà addestrato.

In [27]:
# inizializziamo il vettorizzatore
vectorizer = TfidfVectorizer(sublinear_tf=True, min_df=5, max_df=0.95)
# fit_transform applica il TF-IDF ai testi puliti - salviamo la matrice di vettori in X
data_vectorized = vectorizer.fit_transform(df['tokenized_sents'])

In [28]:
len(vectorizer.get_feature_names_out())

5222

In [29]:
vectorizer.vocabulary_

{'iprp': 2383,
 'weepvbp': 5029,
 'forin': 1795,
 'bevbz': 460,
 'deadjj': 1042,
 'ohuh': 3141,
 'weepjj': 5024,
 'adonaisnnp': 63,
 'thoughin': 4613,
 'ourprp': 3195,
 'tearnns': 4520,
 'notrb': 3073,
 'thedt': 4564,
 'frostnn': 1848,
 'whichwdt': 5060,
 'sorb': 4220,
 'dearjj': 1047,
 'adt': 69,
 'headnn': 2095,
 'andcc': 166,
 'thouvb': 4626,
 'sadjj': 3857,
 'fromin': 1844,
 'alldt': 125,
 'yearnns': 5195,
 'toto': 4692,
 'mournvb': 2940,
 'lossnn': 2665,
 'thynn': 4651,
 'obscurejj': 3094,
 'teachvb': 4517,
 'themprp': 4574,
 'thinevb': 4593,
 'ownjj': 3215,
 'sorrownn': 4221,
 'sayvbp': 3892,
 'within': 5131,
 'meprp': 2809,
 'tillvb': 4662,
 'darevbz': 1027,
 'pastnnp': 3275,
 'hisprp': 2165,
 'fatenn': 1631,
 'famenn': 1603,
 'shallmd': 4021,
 'bevb': 455,
 'andt': 168,
 'echonn': 1370,
 'lightjj': 2578,
 'untojj': 4822,
 'eternitynn': 1476,
 'iinnp': 2251,
 'wherennp': 5055,
 'wertnn': 5041,
 'thounn': 4619,
 'mightynn': 2829,
 'mothernnp': 2925,
 'whenwrb': 5053,
 'heprp': 21

In [30]:
X=data_vectorized.toarray()

In [31]:
Y=np.empty((len(df),1))
Y

array([[0.00000000e+000],
       [6.90314434e-310],
       [6.90314434e-310],
       ...,
       [6.90313833e-310],
       [6.90313833e-310],
       [6.90313833e-310]])

In [32]:
#trasformazione delle classi targhet da stringhe a numeri
authors=df["author"].unique() #lista/insieme degli autori

targets=np.array(df["author"])

for i in range (len(df)):
  Y[i]=np.where(authors == targets[i])[0][0]

Y=Y.astype("int")
Y

array([[0],
       [0],
       [0],
       ...,
       [4],
       [4],
       [4]])

In [33]:
X.shape

(11944, 5222)

In [35]:
Y.shape

(11944, 1)

In [34]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3)

In [61]:
print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)

(16088, 8026) (6895, 8026)
(16088, 1) (6895, 1)


In [62]:
print("Xtrain",type(X_train))
print("Ytrain",type(Y_train))
print("Xtest",type(X_test))
print("Ytest",type(Y_test))
print("Xtrain",X_train.dtype)
print("Ytrain",Y_train.dtype)
print("Xtest",X_test.dtype)
print("Ytest",Y_test.dtype)


Xtrain <class 'numpy.ndarray'>
Ytrain <class 'numpy.ndarray'>
Xtest <class 'numpy.ndarray'>
Ytest <class 'numpy.ndarray'>
Xtrain float64
Ytrain int64
Xtest float64
Ytest int64


**Trasformazione degli array numpy in tensori pytorch**

In [36]:
import torch

X_train = torch.from_numpy(X_train).float()
Y_train = torch.from_numpy(Y_train).float()

X_test = torch.from_numpy(X_test).float()
Y_test = torch.from_numpy(Y_test).float()

In [47]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

torch.Size([16088, 8013])
torch.Size([16088, 1])
torch.Size([6895, 8013])
torch.Size([6895, 1])


# **Creazione della rete neurale feed-forward**

In [37]:
#il numero di neuroni viene fatto corrispondere al numero di colonne di un vettore rappresentante un dato
#cioè in base al numero di termini che hanno una tfidf maggiore di 0

len(X_train[0])

input_layer_neurons=len(X_train[0])
output_layer_neurons=5

print(input_layer_neurons)
print(output_layer_neurons)

5222
5


In [38]:
import torch.nn as nn
import torch.nn.functional as fn

#creazione della classe Net
#Net è una sottoclasse di nn.Module quindi eredita tutti i suoi metodi e attributi
class Net(nn.Module):
  def __init__(self): #costruttore della classe
    super().__init__()  #viene richiamato il costruttore della classe nn.Module

    #definizione dei layers(quindi degli strati della rete che avranno un numero di ingressi e un numero di uscite determinati)
    #nn.linear applica una trasformazione lineare ai dati
    self.fc1=nn.Linear(input_layer_neurons,124)  #Input Layer : ha 28*28 ingressi(quindi 784 neuroni) e 64 uscite possibili(cioé gli ingressi del layer successivo saranno 64)
    self.fc2=nn.Linear(124,64)     #Hidden Layer 1: ha 64 ingressi e 64 uscite
    #self.fc3=nn.Linear(64,64)     #Hidden Layer 2: ha 64 ingressi e 64 uscite
    self.fc4=nn.Linear(64,output_layer_neurons)     #Output Layer : ha 64 ingressi e 10 uscite perché le classi possibili sono 10
  
  #definizione della logica secondo cui i dati attraversano i vari layer
  #ad ogni dato quando attraversa ogni neurone viene applicata la funzione relu
  def forward(self,x):
    x=fn.relu(self.fc1(x))
    x=fn.relu(self.fc2(x))
    #x=fn.relu(self.fc3(x))
    x=self.fc4(x)

    #return x
    return fn.log_softmax(x,dim=1)  #ritorna la distribuzione probabilistica dell'output della rete

In [44]:
net=Net() #istanziazione della rete neurale

# Caricamento dei dati(comprende suddivisione in batch)

In [45]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 10

train_data = TensorDataset(X_train,Y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(X_test,Y_test)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# FASE DI TRAINING

In [46]:
import torch

if torch.cuda.is_available():
  device = torch.device("cuda")
  print("gpu available")
else:
  device = torch.device("cpu")

gpu available


In [47]:
import torch.optim as optim

#implementazione della loss-function o funzione di costo
#la loss-function calcola quanto gli output della rete neurale si discostano dai valori reali
#il risultato di una loss-function può essere uno scalare(entropia incrociata...) oppure un vettore(one-hot array)

loss_function=nn.CrossEntropyLoss();

#implementazione dell'ottimizzatore 
#l'ottimizzatore si occupa di modificare i pesi dei collegamenti della rete in modo che questa restituisca un output che si avvicini il più possibile al valore corretto
#net.parameters() restituisce un tensore che rappresenta i parametri della rete ossia tutti i pesi della rete
#il learning rate (lr) è un valore che dice di quanto devono essere aggiustati i pesi a ogni iterazione
#Adam è un ottimizzatore che implementa la discesa stocastica del gradiente (Stochastic Gradient Descent)

optimizer=optim.Adam(net.parameters(),lr=0.001)

#l'epoch è rappresenta l'intera fase in cui al modello sono stati passati tutti i dati del trainset suddivisi in batches
#se si effettuano poche epoche il modello non si addestra bene o meglio non aggiusta i propri pesi nel modo migliore
#se si effettuano troppe epoche il modello rischia l'overfitting (si adegua troppo al training set e perde la capacità di generalizzare)

for epoch in range(3):
  for data in train_data:
    X, y = data #suddividiamo l'esempio in features (X) e classe di appartenenza (y)
    #print(X)
    #print(y)
    net.zero_grad() #setta il gradiente a 0
    output=net(X.view(-1,input_layer_neurons))  #ad ogni passo alla rete viene fornito come input l'ouput dell'iterazione precedente
    #print(output)
    loss=fn.nll_loss(output,y.long())  #calcola la loss-function sull'output della rete al passo corrente rispetto al target reale
    loss.backward() #l'errore commesso viene propagato verso ogni collegamento dallo strato di output agli strati più interni(back-propagation)
    optimizer.step()  #basandosi sui valori riportati dal gradiente effettua l'aggiustamento dei parametri della rete in maniera opportuna  
  print(loss) #stampa la loss function dopo ogni epoca

tensor(2.2242, grad_fn=<NllLossBackward0>)
tensor(2.6206, grad_fn=<NllLossBackward0>)
tensor(0.8268, grad_fn=<NllLossBackward0>)


In [48]:
correct=0 #numero di esempi su cui la rete ha predetto in maniera corretta
total=0 #numero di esempi su cui la rete ha effettuato una predizione

with torch.no_grad(): #nella fase di test non dobbiamo aggiustare i pesi o calcolare la loss function percui non ci interessa il gradiente
  for data in test_data:
    X, y = data
    output=net(X.view(-1,input_layer_neurons))
    #print(y)
    for idx, i in enumerate(output):
      #print(torch.argmax(i),y[idx])
      if(torch.argmax(i)==y[idx]):
        correct+=1
      total+=1

print("ACCURACY: ",round(correct/total,3)*100,"%")

ACCURACY:  70.3 %


10- 61.19,61.8-----62.8%
3- 77.5